<a href="https://colab.research.google.com/github/cathieG/CodeT5/blob/main/colab/CSCI420_Lab02_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run for dependencies

!pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124
!pip install transformers datasets evaluate -q
!pip install evaluate
!pip install sacrebleu
! pip install tree_sitter==0.2.0
! git clone -q https://github.com/microsoft/CodeXGLUE.git

Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 86.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Import all the packages

import pandas as pd
import numpy as np
from datasets import Dataset
import re
import os
import torch
import evaluate
import io
import tempfile
import zipfile

from google.colab import files, drive

In [ ]:
# Used for saving files to Google Drive. Not necessary
drive.mount('/content/drive')

In [ ]:
# Same as above
uploaded = files.upload()

In [ ]:
# Read in the csv files that I flattened.

df_t_flat = pd.read_csv("flattened_train.csv")
df_test_flat = pd.read_csv("flattened_test.csv")
df_v_flat = pd.read_csv("flattened_valid.csv")

In [ ]:
# Builds the model that will be used

from transformers import T5ForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import RobertaTokenizer
from datasets import DatasetDict
from transformers import TrainingArguments, Trainer
from transformers import EarlyStoppingCallback

model_checkpoint = "Salesforce/codet5-small"
# model_checkpoint = "seven_epoch_checkpoint"
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)

tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)
tokenizer.add_tokens(["<mask>"]) #Imagine we need an extra token. This line adds the extra token to the vocabulary

model.resize_token_embeddings(len(tokenizer))

Embedding(32100, 512)

In [ ]:
print("Model loaded from:", model.config._name_or_path)

Model loaded from: seven_epoch_checkpoint


In [ ]:
# # This was to test out some code. Ignore.

# sample_train = df_t_flat.iloc[0:1000]
# sample_test = df_test_flat.iloc[0:100]
# sample_valid = df_v_flat.iloc[0:100]

In [ ]:
# Changes the pandas files so that the model can read them

hf_dataset = Dataset.from_pandas(df_t_flat)
hf_testset = Dataset.from_pandas(df_test_flat)
hf_validset = Dataset.from_pandas(df_v_flat)

In [ ]:
# # Ignore

# hf_dataset = Dataset.from_pandas(sample_train)
# hf_testset = Dataset.from_pandas(sample_test)
# hf_validset = Dataset.from_pandas(sample_valid)

In [ ]:
# Makes the code into tokens. Taken from the professor

def preprocess_function(examples):
    inputs = examples["flattened_code"]
    targets = examples["target_block"]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=256, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# Tokenizes them

tokenized_train = hf_dataset.map(preprocess_function, batched=True)
tokenized_testset = hf_testset.map(preprocess_function, batched=True)
tokenized_validset = hf_validset.map(preprocess_function, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
# Puts them into a dataset that can be used in the model

tokenized_datasets = DatasetDict({"train": tokenized_train, "validation": tokenized_validset, "test": tokenized_testset})

In [ ]:
# Checks for GPU. Can ignore.

if torch.cuda.is_available():
    print(f"GPU is available. Using {torch.cuda.get_device_name(0)}")
else:
    print("No GPU detected. Using CPU.")

GPU is available. Using Tesla T4


In [ ]:
# Defines "device", which is necessary for the testing part.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
# Can ignore

output_dir = "/content/drive/MyDrive/codet5-finetuned"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Defines all the arguments for training

training_args = TrainingArguments(
    output_dir="codet5-finetuned",
    report_to="none",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=7,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_total_limit=2,
    logging_steps=100,
    logging_strategy="epoch",
    push_to_hub=False,
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

<ipython-input-28-253f186967bb>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:

#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------# ------------------------
# 6. Train the Model
# ------------------------
trainer.train()

# ------------------------
# 7. Evaluate on Test Set
# ------------------------
metrics = trainer.evaluate(tokenized_datasets["test"])
print("Test Evaluation Metrics:", metrics)

# ------------------------
# 8. Test Code Translation
# ------------------------
# input_code = "def add(a, b):\n    return a + b"
# inputs = tokenizer(input_code, return_tensors="pt", padding=True, truncation=True)
# outputs = model.generate(**inputs, max_length=256)
# print("Generated Java Code:\n", tokenizer.decode(outputs[0], skip_special_tokens=True))

Epoch,Training Loss,Validation Loss
1,0.043800,0.035177
2,0.037700,0.033894
3,0.034400,0.033357
4,0.032000,0.032873


In [ ]:
# Can Ignore

# import shutil

# folder_path = '/content/codet5-finetuned'  # Replace with the path to your folder
# zip_name = '/content/test_finetune_lab2.zip'  # This is the name of the zip file
# shutil.make_archive(zip_name.replace('.zip', ''), 'zip', folder_path)


'/content/test_finetune_lab2.zip'

In [ ]:
# Can ignore

# from google.colab import files
# files.download(zip_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Test the code, produces te evaluation metrics, and outputs it to a csv file.

def test_code(csv_file, output_path="evaluation_results.csv"):
  sacrebleu = evaluate.load("sacrebleu")
  predictions = ['x']
  references = ['y']

  flattened_input = []
  correct = []
  targets = []
  code_pred = []
  correct = []
  Bleu_4 = []
  Code_Bleu = []

  i = 0

  for _,row in csv_file.iterrows():
    input = row["flattened_code"]
    target = row["target_block"]
    inputs = tokenizer(input, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(**inputs, max_length=256)
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

    equal_match = prediction == target
    flattened_input.append(input)
    code_pred.append(prediction)
    targets.append(target)
    predictions[0] = prediction
    references[0] = [target]
    correct.append(equal_match)


    # Create temporary files
    with open(f'/content/temp_ref.txt', 'w') as f_ref, open(f'/content/temp_pred.txt', 'w') as f_pred:
        # f_ref.write(targets[i] + '\n')
        # f_pred.write(predictions[i] + '\n')
        f_ref.write(targets[i] + '\n')
        f_pred.write(code_pred[i] + '\n')

    # Call the calc_code_bleu script
    result = os.popen(
        'cd /content/CodeXGLUE/Code-Code/code-to-code-trans/evaluator/CodeBLEU/ && '
        'python calc_code_bleu.py --refs /content/temp_ref.txt '
        '--hyp /content/temp_pred.txt --lang java --params 0.25,0.25,0.25,0.25'
    ).read()

    # Extract the final score
    for line in result.split('\n'):
        if 'CodeBLEU score' in line:
            score_str = str(float(line.split(':')[-1].strip()) * 100)
            Code_Bleu.append(score_str)
            # print(f"[{i}] CodeBLEU: {score_str}")
            break

    results = sacrebleu.compute(predictions=predictions, references=references)
    Bleu_4.append(round(results["score"], 6))




    if i%10 == 0:
      print("This is iteration: "+str(i))
      print("The input is: "+str(input))
      print("The prediction is: "+str(prediction))
      print("The target is: "+str(target))
      print("The prediction is equal to the target? "+str(equal_match))
      print("The CodeBLEU score is: "+str(Code_Bleu[i]))
      print("The BLEU-4 score is: "+str(Bleu_4[i]))
      print()

    i += 1

  results_df = pd.DataFrame({
        "Input": flattened_input,
        "Exact Match": correct,
        "Expected if condition": targets,
        "Predicted if condition": code_pred,
        "CodeBLEU Score": Code_Bleu,
        "BLEU-4 Score": Bleu_4,
    })

  results_df.to_csv(output_path, index=False)




In [ ]:
# Runs the test code

test_code(df_test_flat)